In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

import pyspark as ps

In [2]:
spark = (ps.sql.SparkSession
         .builder
         .master('local[4]')
         .appName('constructionPermitsAustin')
         .getOrCreate()
        )
sc = spark.sparkContext

In [3]:
sc

<SparkContext master=local[4] appName=constructionPermitsAustin>

In [5]:
df_permits = spark.read.csv('../data/Issued_Construction_Permits.csv',
                    header=True,       # use headers or not
                    sep=",",           # char for separation
                    inferSchema=True)  # do we infer schema or not ?

# prints the schema
df_permits.printSchema()

# some functions are still valid
print(f'line count: {df_permits.count()}')

# show the table in a oh-so-nice format
df_permits.show()

root
 |-- Permit Type: string (nullable = true)
 |-- Permit Type Desc: string (nullable = true)
 |-- Permit Num: string (nullable = true)
 |-- Permit Class Mapped: string (nullable = true)
 |-- Permit Class: string (nullable = true)
 |-- Work Class: string (nullable = true)
 |-- Condominium: string (nullable = true)
 |-- Project Name: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- TCAD ID: string (nullable = true)
 |-- Property Legal Description: string (nullable = true)
 |-- Applied Date: string (nullable = true)
 |-- Issued Date: string (nullable = true)
 |-- Day Issued: string (nullable = true)
 |-- Calendar Year Issued: integer (nullable = true)
 |-- Fiscal Year Issued: integer (nullable = true)
 |-- Issued In Last 30 Days: string (nullable = true)
 |-- Issuance Method: string (nullable = true)
 |-- Status Current: string (nullable = true)
 |-- Status Date: string (nullable = true)
 |-- Expires Date: string (nullable = true)
 |-- Completed Date: string (nu

In [6]:
df_permits.createOrReplaceTempView('permits')

In [13]:
permits_2019 = spark.sql('SELECT * from permits WHERE `Calendar Year Issued` = 2019')

In [14]:
permits_2020 = spark.sql('SELECT * from permits WHERE `Calendar Year Issued` = 2020')

In [20]:
permits_2020_pd = permits_2020.toPandas()

In [21]:
permits_2020_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69317 entries, 0 to 69316
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Permit Type                   69317 non-null  object 
 1   Permit Type Desc              69317 non-null  object 
 2   Permit Num                    69317 non-null  object 
 3   Permit Class Mapped           69317 non-null  object 
 4   Permit Class                  69317 non-null  object 
 5   Work Class                    69317 non-null  object 
 6   Condominium                   53516 non-null  object 
 7   Project Name                  69317 non-null  object 
 8   Description                   69317 non-null  object 
 9   TCAD ID                       68719 non-null  object 
 10  Property Legal Description    61782 non-null  object 
 11  Applied Date                  69314 non-null  object 
 12  Issued Date                   69317 non-null  object 
 13  D

In [58]:
permits_2020_pd['Location'].value_counts()

(30.26763064, -97.74596464)    124
(30.1996283, -97.73520938)     124
(30.27942924, -97.74159221)    100
(30.25349256, -97.74794018)     97
(30.35444794, -97.68546322)     94
                              ... 
(30.40350827, -97.614915)        1
(30.40544549, -97.621067)        1
(30.47071803, -97.68254062)      1
(30.25071484, -97.76613519)      1
(30.3039993, -97.71324492)       1
Name: Location, Length: 28112, dtype: int64

In [59]:
permits_2020_pd_cleaned = permits_2020_pd.drop(columns=['Project Name',
                                                        'TCAD ID',
                                                        'Property Legal Description',
                                                        'Issued In Last 30 Days',
                                                        'Issuance Method',
                                                        'Original Address 1',
                                                        'Original City',
                                                        'Original State',
                                                        'Jurisdiction',
                                                        'Link',
                                                        'Project ID',
                                                        'Master Permit Num',
                                                        'Location',
                                                        'Contractor Phone',
                                                        'Contractor Phone',
                                                        'Contractor Phone',
                                                        'Contractor Address 1',
                                                        'Contractor Address 2',
                                                        'Contractor City',
                                                        'Contractor Zip',
                                                        'Applicant Phone',
                                                        'Applicant Address 1',
                                                        'Applicant Address 2',
                                                        'Applicant City',
                                                        'Applicant Zip',
                                                        'Total Lot SQFT'])

permits_2020_pd_cleaned

,Permit Type,Permit Type Desc,Permit Num,Permit Class Mapped,Permit Class,Work Class,Condominium,Description,Applied Date,Issued Date,...,Council District,Latitude,Longitude,Location,Contractor Trade,Contractor Company Name,Contractor Full Name,Applicant Full Name,Applicant Organization,Certificate Of Occupancy
0,PP,Plumbing Permit,2020-001010 PP,Residential,Residential,Irrigation,None,new irrigation system install,2020/01/03,2020/01/09,...,NaN,30.164819,-97.632821,"(30.16481871, -97.63282113)",None,None,None,None,None,None
1,PP,Plumbing Permit,2020-003546 PP,Residential,Residential,Irrigation,None,irrigation installation,2020/01/09,2020/01/10,...,6.0,30.501049,-97.782213,"(30.50104899, -97.78221326)",None,None,None,None,None,None
2,PP,Plumbing Permit,2020-015357 PP,Residential,Residential,Irrigation,None,IRRIGATION FOR LAWN,2020/01/31,2020/02/04,...,6.0,30.500805,-97.747047,"(30.500805, -97.74704689)",None,None,None,None,None,None
3,PP,Plumbing Permit,2020-015365 PP,Residential,Residential,Irrigation,None,IRRIGATION FOR LAWN,2020/01/31,2020/02/04,...,6.0,30.503280,-97.747578,"(30.50328, -97.74757756)",None,None,None,None,None,None
4,EP,Electrical Permit,2020-002321 EP,Commercial,Sign Permit,Wall,None,install channel letters Floor Decor Sign D,2020/01/28,2020/01/28,...,5.0,30.162663,-97.787009,"(30.1626626, -97.78700858)",None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69312,BP,Building Permit,1998-011758 BP,Residential,R- 434 Addition & Alterations,Addition,No,Add 1st Fl Familyroom 2nd Fl Master Bedroom,1998/04/10,2020/09/18,...,9.0,30.293143,-97.754427,"(30.29314281, -97.75442744)",General Contractor,None,Elise Schram,None,None,Yes
69313,BP,Building Permit,2005-022645 BP,Commercial,C- 329 Com Structures Other Than Bldg,New,No,New Swimming Pool,2005/04/01,2020/07/27,...,10.0,30.332960,-97.803205,"(30.33296039, -97.80320539)",General Contractor,"Ocean Quest Pools, Inc.",None,None,None,No
69314,BP,Building Permit,2020-185483 BP,Residential,R- 101 Single Family Houses,New,Yes,1604 Unit 75 Aberdeen 1799 AR New Residentia...,2020/11/24,2020/12/08,...,1.0,30.372453,-97.654784,"(30.37245306, -97.6547841)",General Contractor,"Prominence Homes, LLC ***MAIN",Bryan Rome,Bryan Rome,"Prominence Homes, LLC ***MAIN",Yes
69315,BP,Building Permit,2020-185481 BP,Residential,R- 101 Single Family Houses,New,Yes,1600 Unit 73 Alexander1907 AR New 2 Story Res...,2020/11/24,2020/12/08,...,1.0,30.372277,-97.654773,"(30.37227681, -97.65477314)",General Contractor,"Prominence Homes, LLC ***MAIN",Bryan Rome,Bryan Rome,"Prominence Homes, LLC ***MAIN",Yes


In [60]:
permits_2020_pd_cleaned['Applied Date'] = pd.to_datetime(permits_2020_pd_cleaned['Applied Date'])
permits_2020_pd_cleaned['Issued Date'] = pd.to_datetime(permits_2020_pd_cleaned['Issued Date'])
permits_2020_pd_cleaned['Status Date'] = pd.to_datetime(permits_2020_pd_cleaned['Status Date'])
permits_2020_pd_cleaned['Expires Date'] = pd.to_datetime(permits_2020_pd_cleaned['Expires Date'])
permits_2020_pd_cleaned['Completed Date'] = pd.to_datetime(permits_2020_pd_cleaned['Completed Date'])

In [61]:
permits_2020_pd_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69317 entries, 0 to 69316
Data columns (total 45 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Permit Type                   69317 non-null  object        
 1   Permit Type Desc              69317 non-null  object        
 2   Permit Num                    69317 non-null  object        
 3   Permit Class Mapped           69317 non-null  object        
 4   Permit Class                  69317 non-null  object        
 5   Work Class                    69317 non-null  object        
 6   Condominium                   53516 non-null  object        
 7   Description                   69317 non-null  object        
 8   Applied Date                  69314 non-null  datetime64[ns]
 9   Issued Date                   69317 non-null  datetime64[ns]
 10  Day Issued                    69317 non-null  object        
 11  Calendar Year Issued        